Reference:

[**Introduction to LangChain for Agentic AI**](https://courses.analyticsvidhya.com/courses/take/introduction-to-langchain-for-agentic-ai/lessons/61748853-course-introduction) 

## Install OpenAI, and LangChain dependencies

In [1]:
# !pip install -qq langchain
# !pip install -qq langchain-openai
# !pip install -qq langchain-community
# !pip install -qq langchain-chroma

## Enter Open AI API Key

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

True

## Load Connection to LLM

Here we create a connection to ChatGPT to use later in our chains

In [3]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name='gpt-4o-mini', temperature=0)

## Multi-user Window-based Conversation Chains with persistence - SQLChatMessageHistory

The beauty of `SQLChatMessageHistory` is that we can store separate conversation histories per user or session which is often the need for real-world chatbots which will be accessed by many users at the same time. Instead of in-memory we can store it in a SQL database which can be used to store a lot of conversations.

We use a `get_session_history` function which is expected to take in a `session_id` and return a Message History object. Everything is stored in a SQL database. This `session_id` is used to distinguish between separate conversations, and should be passed in as part of the config when calling the new chain

We also use a `memory_buffer_window` function to only use the top-K last historical conversations before sending it to the LLM, basically our own implementation of `ConversationBufferWindowMemory`


- `ChatMessageHistory`: This is a class in LangChain that stores a sequence of chat messages (such as HumanMessage, AIMessage, SystemMessage) for a conversation. It allows you to append new messages and retrieve the full message history, which is useful for maintaining conversational context.

- `BaseChatMessageHistory`: This is an abstract base class that defines the interface for chat message history storage backends. It specifies methods like add_message, get_messages, and clear, which concrete implementations (like ChatMessageHistory, RedisChatMessageHistory, etc.) must provide.

- `RunnableWithMessageHistory`: This is a wrapper for a Runnable (such as a chain or LLM) that automatically manages message history for each session or user. It uses a function (like get_session_history) to retrieve the appropriate message history object based on a session ID, and injects the history into the chain's input. This enables multi-user or multi-session conversational experiences.

- `MessagesPlaceholder`: This is a special prompt template variable in LangChain that acts as a placeholder for a list of messages (e.g., the conversation history). When rendering a prompt, MessagesPlaceholder is replaced with the actual messages from history, allowing the LLM to see the full conversation context.


In [4]:
# removes the memory database file - usually not needed
# you can run this only when you want to remove all conversation histories
# !rm memory.db

In [5]:
from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough

# used to retrieve conversation history from database
# based on a specific user or session ID
def get_session_history_db(session_id):
    return SQLChatMessageHistory(session_id, "sqlite:///memory.db")

# prompt to load in history and current input from the user
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Act as a helpful AI Assistant"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{human_input}"),
    ]
)

# create a memory buffer window function to return the last K conversations
def memory_buffer_window(messages, k=2):
    return messages[-(k+1):]

# create a basic LLM Chain which only sends the last K conversations per user
llm_chain = (
    RunnablePassthrough.assign(history=lambda x: memory_buffer_window(x["history"]))
      |
    prompt_template
      |
    chatgpt
)

In [6]:
# create a conversation chain which can load memory based on specific user or session id
conv_chain = RunnableWithMessageHistory(
    llm_chain,
    get_session_history_db,
    input_messages_key="human_input",
    history_messages_key="history",
)

# create a utility function to take in current user input prompt and their session ID
# streams result live back to the user from the LLM
def chat_with_llm(prompt: str, session_id: str):
    for chunk in conv_chain.stream({"human_input": prompt},
                                   {'configurable': { 'session_id': session_id}}):
        print(chunk.content, end="")

In [7]:
user_id = 'jim001'
prompt = "Hi can you tell me which is the fastest animal?"
chat_with_llm(prompt, user_id)

/Users/sourav.banerjee/Documents/Codebases/2. AI ENGINEERING/LangChain_Demystified/.venv/lib/python3.13/site-packages/langchain_core/runnables/history.py:606: LangChainDeprecationWarning: `connection_string` was deprecated in LangChain 0.2.2 and will be removed in 1.0. Use connection instead.
  message_history = self.get_session_history(


The fastest animal in the world is the peregrine falcon. When in a dive, it can reach speeds of over 240 miles per hour (386 kilometers per hour). If you're considering land animals, the cheetah holds the title for the fastest, capable of running speeds up to 60-70 miles per hour (97-113 kilometers per hour) in short bursts. For the fastest animal in water, the black marlin is often cited, reaching speeds of up to 82 miles per hour (132 kilometers per hour).

In [8]:
prompt = "what about the slowest animal?"
chat_with_llm(prompt, user_id)

The slowest animal is often considered to be the three-toed sloth. It moves at an average speed of about 0.24 kilometers per hour (0.15 miles per hour) when climbing trees. In water, they can swim slightly faster, but their overall movement is very slow due to their low metabolic rate and energy-efficient lifestyle. Another contender for the title of slowest animal is the garden snail, which moves at a speed of about 0.03 miles per hour (0.048 kilometers per hour).


#### ![Chat Message Memory Flow](../images/sqllite.png)

In [9]:
prompt = "what about the largest animal?"
chat_with_llm(prompt, user_id)

The largest animal on Earth is the blue whale (*Balaenoptera musculus*). Adult blue whales can reach lengths of up to 100 feet (30 meters) or more and can weigh as much as 200 tons (approximately 181 metric tonnes). These magnificent marine mammals are not only the largest animals alive today but are also believed to be the largest animals to have ever existed on Earth. Their size is truly remarkable, with hearts that can weigh as much as a small car and tongues that can weigh as much as an elephant!

In Below cell we will see the LLM always will get last 2 message in its history as we have kept window = 2

In [10]:
prompt = "what topics have we discussed, show briefly as bullet points"
chat_with_llm(prompt, user_id)

Sure! Here are the topics we've discussed so far:

- The slowest animal (three-toed sloth and garden snail)
- The largest animal (blue whale)

In [11]:
user_id = 'john005'
prompt = "Explain AI in 3 bullets to a child"
chat_with_llm(prompt, user_id)

Sure! Here are three simple points to explain AI to a child:

1. **Smart Helpers**: AI is like a smart robot or computer that can help us do things, like answering questions or playing games, by learning from lots of information.

2. **Learning from Experience**: Just like you learn new things by practicing, AI learns by looking at examples and figuring out patterns, so it can get better over time.

3. **Talking and Understanding**: AI can understand what we say and even talk back to us, like a friend who knows a lot and can help us find answers or tell stories!

In [12]:
prompt = "Now do the same for Generative AI"
chat_with_llm(prompt, user_id)

Of course! Here are three simple points to explain Generative AI to a child:

1. **Creative Robot**: Generative AI is like a creative robot that can make new things, like pictures, stories, or music, all by itself!

2. **Learning from Examples**: It learns by looking at lots of examples, just like how you get ideas for your drawings or stories by seeing what others have made.

3. **Imagination in Action**: When you ask Generative AI to create something, it uses its imagination to come up with fun and unique ideas, almost like magic!

In [13]:
prompt = "Now do the same for machine learning"
chat_with_llm(prompt, user_id)

Sure! Here are three simple points to explain machine learning to a child:

1. **Learning Like You**: Machine learning is when computers learn from examples, just like you learn new things by practicing and trying different activities.

2. **Finding Patterns**: It helps computers find patterns in information, like how you notice that some animals have stripes or spots, so they can make smart guesses about new things.

3. **Getting Better Over Time**: The more examples the computer sees, the better it gets at understanding and making decisions, kind of like how you get better at a game the more you play it!

In [14]:
prompt = "what topics have we discussed, show briefly as bullet points"
chat_with_llm(prompt, user_id)

Sure! Here are the topics we've discussed so far:

- **Generative AI**
  - Creative robot that makes new things
  - Learns from examples
  - Uses imagination to create unique ideas

- **Machine Learning**
  - Computers learn like humans do
  - Finds patterns in information
  - Improves over time with more examples